In [1]:
import tensorflow as tf
import pathlib
from keras_preprocessing import image
import numpy as np
import PIL
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.python.keras import regularizers

In [2]:
folders = tf.io.gfile.glob("data")
img = tf.io.gfile.glob("data\*\*.jpg")
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  folders[0],
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  )

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  folders[0],
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  )

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
num_classes = 2

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001)),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu',kernel_regularizer=regularizers.l2(0.001)),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.001)),
  layers.Dropout(0.7),
  layers.Dense(num_classes)
])
class_names = train_ds.class_names

Found 1376 files belonging to 2 classes.
Using 1101 files for training.
Found 1376 files belonging to 2 classes.
Using 275 files for validation.


In [3]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])
filepath="weights.best.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
callbacks_list = [checkpoint]
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5,
  callbacks=callbacks_list 
)

Epoch 1/5
35/35 [==============================] - 53s 1s/step - loss: 0.9928 - accuracy: 0.5402 - val_loss: 0.6874 - val_accuracy: 0.8909
Epoch 2/5
35/35 [==============================] - 31s 881ms/step - loss: 0.5550 - accuracy: 0.8603 - val_loss: 0.2669 - val_accuracy: 0.9527
Epoch 3/5
35/35 [==============================] - 31s 885ms/step - loss: 0.2638 - accuracy: 0.9588 - val_loss: 0.2522 - val_accuracy: 0.9382
Epoch 4/5
35/35 [==============================] - 32s 891ms/step - loss: 0.2506 - accuracy: 0.9561 - val_loss: 0.2206 - val_accuracy: 0.9673
Epoch 5/5
35/35 [==============================] - 31s 886ms/step - loss: 0.1819 - accuracy: 0.9741 - val_loss: 0.2619 - val_accuracy: 0.9527


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 87, 87, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 20, 32)        0

In [5]:
model.built = True

In [6]:
model.load_weights("weights.best.hdf5")

In [7]:
images = image.load_img("dataset/User.22.jpg", target_size=(img_height, img_width))    
x = image.img_to_array(images)
#x = tf.image.rgb_to_grayscale(x)
x = np.expand_dims(x, axis=0)
#x = x/255.0
arr = model.predict(x)
print(class_names[np.argmax(arr)])

without_mask


In [9]:
import cv2
import sys
global new_array 
cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

video_capture = cv2.VideoCapture(0)
count = 0
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    #img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        frame,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
        #flags=cv2.CV_HAAR_SCALE_IMAGE
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        saved_img = frame[y:y+h, x:x+w]
        count += 1
        cv2.imwrite("dataset3/User." +  
                str(count) + ".jpg", saved_img)
        images = image.load_img("dataset3/User." +  str(count) + ".jpg",target_size=(180, 180))   
        #cv2.imwrite("dataset3/User." +  
            #        str(count) + ".jpg", images)
        #images = cv2.resize(saved_img,dim=(180,180))
        face_image = image.img_to_array(images)
        face_image = np.expand_dims(face_image, axis=0)
        arr = model.predict(face_image)
        new_array = face_image
        label = np.argmax(arr)
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255, 0, 0))
        cv2.rectangle(frame,(x,y-40),(x+w,y),(255, 0, 0))
        cv2.putText(frame,class_names[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('Video', frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [22]:
new_array

array([[[[  0., 255.,   0.],
         [  0., 255.,   0.],
         [  0., 255.,   0.],
         ...,
         [  0., 255.,   0.],
         [  0., 255.,   0.],
         [  0., 255.,   0.]],

        [[  0., 255.,   0.],
         [  4., 247.,   3.],
         [ 20., 218.,  15.],
         ...,
         [ 27., 233.,  32.],
         [ 28., 233.,  33.],
         [  2., 253.,   2.]],

        [[  0., 255.,   0.],
         [ 20., 217.,  15.],
         [ 95.,  76.,  70.],
         ...,
         [132., 149., 155.],
         [135., 150., 158.],
         [  9., 248.,  11.]],

        ...,

        [[  0., 255.,   0.],
         [  8., 208.,   7.],
         [ 36.,  31.,  34.],
         ...,
         [128., 136., 141.],
         [130., 139., 143.],
         [  9., 247.,  10.]],

        [[  0., 255.,   0.],
         [  7., 208.,   7.],
         [ 35.,  31.,  33.],
         ...,
         [141., 150., 153.],
         [132., 142., 145.],
         [  9., 247.,  10.]],

        [[  0., 255.,   0.],
       

In [37]:
#images = im.open("dataset3/User.107.jpg")
face_image = image.load_img("dataset3/User.108.jpg")  
face_image = np.expand_dims(face_image, axis=0)
#print(face_image)
arr = model.predict(face_image)
#new_array = face_image
label = class_names[np.argmax(arr)]
print(label)

without_mask
